In [5]:
import os
import sys

import numpy as np
import numpy_financial as npf

from scipy.optimize import newton
from datetime import datetime


Modified IRR reflects the incorporate reinvestment by assuming the postive cash flows to LPs are reinvested at more realistic expected return. so, cost of capital and reinvestment rate (usually average PE asset class return or public benchmark)

But still, MIRR does not directly compare to liquid asset classes where valuations and returns are easily determined through a daily mark-to-market.

In [6]:
from scipy.optimize import newton
import numpy_financial as npf
from datetime import datetime

def calculate_days(dates):
    """Calculate days between each date and the first date."""
    start_date = datetime.strptime(dates[0], "%Y-%m-%d")
    days = [(datetime.strptime(date, "%Y-%m-%d") - start_date).days for date in dates]
    return days

def objective_function(rate, pv_negative, fv_positive, days):
    """
    Objective function for the Newton method, calculating the NPV of modified cash flows at 'rate'.
    """
    npv = npf.npv(rate, [pv_negative] + [0]*(len(days)-2) + [fv_positive])
    return npv

def mirr_newton(cashflows, cost_of_capital, reinvestment_rate, dates):
    """
    Calculate MIRR using the Newton method, considering cost of capital and reinvestment rate.
    """
    days = calculate_days(dates)
    years = [day / 360 for day in days]  # Convert days to years for each cash flow
    
    # Separate positive and negative cash flows
    positive_cashflows = np.array(cashflows)
    positive_cashflows[positive_cashflows < 0] = 0
    negative_cashflows = np.array(cashflows)
    negative_cashflows[negative_cashflows > 0] = 0
    
    # Calculate the Present Value (PV) of negative cashflows at the financing rate
    pv_negative = npf.npv(cost_of_capital, negative_cashflows)
    
    # Calculate the Future Value (FV) of positive cashflows at the reinvestment rate
    fv_positive = npf.fv(reinvestment_rate, max(years), 0, -sum(positive_cashflows))
    
    # Use the Newton method to solve for MIRR
    mirr_result = newton(func=objective_function, x0=0.1, args=(pv_negative, fv_positive, days))
    
    return mirr_result


In [7]:
# Using the updated parameters for calculation
cost_of_capital = 0.08
reinvestment_rate = 0.12
dates = ['2018-01-01', '2019-12-31', '2020-12-31', '2021-12-31']
cashflows = [-100, -20, 50, 100]

0.2607737463192273